In [134]:
from decouple import config, AutoConfig

In [135]:
config = AutoConfig(search_path='.')

In [136]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [137]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [138]:
ALPHA_VANTAGE_API_KEY

'ANIJLNK3PVWBJLR6'

In [108]:
# import requests

# params = {
#     "api_key" : ALPHA_VANTAGE_API_KEY,
#     "ticker": "AAPL",
#     "function": "TIME_SERIES_INTRADAY",
#     "interval": "5min",
# }

# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}'.format(
#     **params
# )

# r = requests.get(url)
# data = r.json()

In [109]:
# dataset_key = [x for x in list(data.keys()) if not x == "Meta Data"][0]

In [110]:
# results = data[dataset_key]
# print(results)

In [111]:
# timestamp_str = list(results.keys())[0]
# timestamp_str

In [112]:
# result_val = results.get(timestamp_str)

In [113]:
# import pytz
# from datetime import datetime
# from decimal import Decimal

# def transform_alpha_vantage_result(timestamp_str, result):
#     timestamp_format = "%Y-%m-%d %H:%M:%S"
#     eastern = pytz.timezone("US/Eastern")
#     utc = pytz.utc

#     utc_timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)

#     #unix_timestamp = result.get('t') / 1000.0
#     #utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))

    
#     return {
#         'open_price': Decimal(result['1. open']),
#         'close_price': Decimal(result['4. close']),
#         'high_price': Decimal(result['2. high']),
#         'low_price': Decimal(result['3. low']),
#         'number_of_trades': None,
#         'volume': int(result['5. volume']),
#         'volume_weighted_average': None,
#         'time': utc_timestamp,
#     }

In [114]:
# transform_alpha_vantage_result(timestamp_str, result_val)

In [116]:
# for timestamp_str in results.keys():
#    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
#    print(new_data)

In [ ]:
import pytz
from typing import Literal
from datetime import datetime
from dataclasses import dataclass
from decimal import Decimal
from urllib.parse import urlencode

ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

def transform_alpha_vantage_result(timestamp_str, result):
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc

    utc_timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': utc_timestamp,
    }

@dataclass
class AlphaVantageAPIClient:
    ticker:str = "AAPL"
    function: Literal["TIME_SERIES_INTRADAY"] = "TIME_SERIES_INTRADAY"
    interval: Literal["1min", "5min", "15min", "30min", "60min"] = "5min"
    month:str = "2024-01"
    api_key:str = ""

    def get_api_key(self):
        return self.api_key or ALPHA_VANTAGE_API_KEY
            
    def get_params(self):
        return {
            "apikey": self.get_api_key(),
            "symbol": self.ticker,
            "function": self.function,
            "interval": self.interval,
            "month": self.month,
        }
    
    def get_headers(self):
        api_key = self.get_api_key()
        return {}

    def generate_url(self, pass_auth=False):
        path = '/query'
        url = f'https://www.alphavantage.co{path}'
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&api_key={api_key}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() #not 200/201 status response
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
        results = data[dataset_key]
        dataset = []
        for timestamp_str in results.keys():
            dataset.append(
                transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
            )
        return dataset

In [155]:
AlphaVantageAPIClient().get_stock_data()

{'2024-01-31 19:55:00': {'1. open': '184.5862', '2. high': '184.8648', '3. low': '184.5763', '4. close': '184.7753', '5. volume': '7522'}, '2024-01-31 19:50:00': {'1. open': '184.5962', '2. high': '184.6260', '3. low': '184.4966', '4. close': '184.5962', '5. volume': '3030'}, '2024-01-31 19:45:00': {'1. open': '184.6559', '2. high': '184.6559', '3. low': '184.5962', '4. close': '184.5962', '5. volume': '3449'}, '2024-01-31 19:40:00': {'1. open': '184.5862', '2. high': '184.6758', '3. low': '184.4966', '4. close': '184.6559', '5. volume': '1002'}, '2024-01-31 19:35:00': {'1. open': '184.6061', '2. high': '184.6658', '3. low': '184.4469', '4. close': '184.5464', '5. volume': '4995'}, '2024-01-31 19:30:00': {'1. open': '184.4961', '2. high': '184.6758', '3. low': '184.3971', '4. close': '184.6758', '5. volume': '3732'}, '2024-01-31 19:25:00': {'1. open': '184.3474', '2. high': '184.5464', '3. low': '184.3474', '4. close': '184.4469', '5. volume': '2533'}, '2024-01-31 19:20:00': {'1. open'

[{'open_price': Decimal('184.5862'),
  'close_price': Decimal('184.7753'),
  'high_price': Decimal('184.8648'),
  'low_price': Decimal('184.5763'),
  'number_of_trades': None,
  'volume': 7522,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 2, 1, 0, 55, tzinfo=<UTC>)}]